In [9]:
import pandas as pd
INPUT = '../data/NRGSTREAM_ERC_UnplannedOutages_06.01.2023-10.01.2023.csv'

In [18]:
# Read the input file, assuming headers are in the 6th row (index 5 in Python)
df = pd.read_csv(INPUT, header=4)

# Extract the relevant columns
df['Outage Start'] = pd.to_datetime(df['Outage Start'], errors='coerce')
df['Outage End'] = pd.to_datetime(df['Outage End'], errors='coerce')

# Drop rows with invalid datetime values
df = df.dropna(subset=['Outage Start', 'Outage End'])

# Filter for Fuel Type containing "coal", "gas", "lignite", or "wood"
df = df[df['Fuel Type'].str.contains('coal|gas|lignite|wood', case=False, na=False)]

# Sort rows
df = df.sort_values(by=['Outage Start', 'Outage End'], ascending=[False, True]).reset_index(drop=True)


In [15]:
# Drop rows with missing values in 'Resource Name' and 'Outage Start'
df = df.dropna(subset=['Resource Name', 'Outage Start']).reset_index(drop=True)

# Create the clean1 column
df['clean1'] = 0
counter = 0
for i in range(len(df)):
    if i > 0 and (
        df.iloc[i]['Resource Name'] == df.iloc[i - 1]['Resource Name'] and
        df.iloc[i]['Outage Start'] == df.iloc[i - 1]['Outage Start']
    ):
        df.loc[i, 'clean1'] = counter
    else:
        counter += 1
        df.loc[i, 'clean1'] = counter
print(df.tail())

     Station Long Name Resource Name    Fuel Type            Outage Type  \
6135               WAP    WAP_WAP_G1  Natural Gas       Forced Extension   
6136               WAP    WAP_WAP_G1  Natural Gas       Forced Extension   
6137               WAP    WAP_WAP_G1  Natural Gas       Forced Extension   
6138               WAP    WAP_WAP_G1  Natural Gas       Forced Extension   
6139          TY_COOKE  TY_COOKE_GT3  Natural Gas  Unavoidable Extension   

      Available MW  Outage MW  Reduction MW        Outage Start  \
6135           169        152            17 2022-09-06 20:11:00   
6136           169        152            17 2022-09-06 20:11:00   
6137           169        152            17 2022-09-06 20:11:00   
6138           169        152            17 2022-09-06 20:11:00   
6139            17          0            17 2022-08-04 00:00:00   

              Outage End                    Reason  clean1  
6135 2023-07-18 22:00:00  Exhaust Problems/Repairs    5334  
6136 2023-07-16 16

In [16]:
# Create the clean2 column
df['clean2'] = False
for i in range(len(df)):
    df.loc[i, 'clean2'] = (
        (i > 0 and df.iloc[i]['clean1'] == df.iloc[i - 1]['clean1']) or
        (i < len(df) - 1 and df.iloc[i]['clean1'] == df.iloc[i + 1]['clean1'])
    )
print(df[df['clean2'] == True])


     Station Long Name Resource Name         Fuel Type            Outage Type  \
22                  SL      SL_SL_G3  Waste/Other Coal    Maintenance Level 2   
23                  SL      SL_SL_G3  Waste/Other Coal  Unavoidable Extension   
42                 THW   THW_THWGT44       Natural Gas  Unavoidable Extension   
43                 THW   THW_THWGT44       Natural Gas  Unavoidable Extension   
59                 PSG   PSG_PSG_ST2       Natural Gas  Unavoidable Extension   
...                ...           ...               ...                    ...   
6134               WAP    WAP_WAP_G1       Natural Gas       Forced Extension   
6135               WAP    WAP_WAP_G1       Natural Gas       Forced Extension   
6136               WAP    WAP_WAP_G1       Natural Gas       Forced Extension   
6137               WAP    WAP_WAP_G1       Natural Gas       Forced Extension   
6138               WAP    WAP_WAP_G1       Natural Gas       Forced Extension   

      Available MW  Outage 